# Analysis of Real Estate Sales in King County, Washington

## Overview

We were tasked with creating a predictive model for home prices in King County, Washington. We elected to focus our model on predicting prices with the goal of helping families that earn roughly the median income for a household in Seattle (~100,000 dollars per year) buy a home that is in their price range and in an area with good schools. We used housing sale data from the King County House Sales dataset, student test score data from Washington State's open data portal, and school address information from the Washington School directory to build our model. We tried a variety of approaches in building our model, and using the strongest of these (r-squared of .71) we created predicted home sale prices for the dataset and compared those predictions to actual sale prices to determine in which zip codes homes are selling for less than we expect, and in which zip codes schools are better than average. Based on these factors, we recommend that families look to purchase housing in the 98004 zip code, which is in Bellevue, WA (10 minutes outside of Seattle by car).

## Business Problem

The Agrawal-Gerber Realty Firm assists families moving to King County, Washington in finding a market-rate home in an area with good schools for their kids. To inform our recommendations, we have created a model to predict the price of homes in King County (which comprises Seattle, Bellevue, Kent, and several other small- to mid-sized cities and towns). Housing in King County is very expensive (the median home in the dataset of sales county sold for 860,000 dollars), so families need to know that their investment represents a good value before they commit to buying a home.

By measuring our predictions against records of actual home sales from 2021 to 2022, we can recommend areas where we believe homes are selling for less than their true value - thus providing families with the ability to buy a home at a good price before the housing market catches up.

## Data Understanding

We used information from three datasets to build our model, all of which are saved in the data folder of our repository:
- Housing sale data from the King County House Sales dataset, which includes the target variable for our models (price) and a wide range of continuous, discrete, and categorical predictor variables, such as zip code, square feet of living space, and more.
- Student test score data from Washington State's open data portal, which we used to build a composite measure of average school quality by zip code to use as a predictor for our model.
- School address information from the Washington School directory to build our model, which we used to match schools to zip codes in the King County House Sales dataset.

To begin, we have to import the necessary libraries for our analysis and the King County House Sales dataset.

In [2]:
# Standard Packages
import pandas as pd
import numpy as np
import markdown

# Viz Packages
import seaborn as sns
import matplotlib.pyplot as plt

# Scipy Stats
import scipy.stats as stats 

# Statsmodel Api
import statsmodels.api as sm
from statsmodels.formula.api import ols

# SKLearn Modules
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures

# Suppress future and deprecation warnings
import warnings
warnings.filterwarnings("ignore", category= FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [3]:
hd = pd.read_csv("data/kc_house_data.csv")
f = open('data/column_names.md', 'r')
desc=markdown.markdown( f.read() )
hd.head(2)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,greenbelt,...,sewer_system,sqft_above,sqft_basement,sqft_garage,sqft_patio,yr_built,yr_renovated,address,lat,long
0,7399300360,5/24/2022,675000.0,4,1.0,1180,7140,1.0,NO,NO,...,PUBLIC,1180,0,0,40,1969,0,"2102 Southeast 21st Court, Renton, Washington ...",47.461975,-122.19052
1,8910500230,12/13/2021,920000.0,5,2.5,2770,6703,1.0,NO,NO,...,PUBLIC,1570,1570,0,240,1950,0,"11231 Greenwood Avenue North, Seattle, Washing...",47.711525,-122.35591


Let's examine some of the basic properties of our dataset.

In [4]:
# check column names
hd.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'greenbelt', 'nuisance', 'view',
       'condition', 'grade', 'heat_source', 'sewer_system', 'sqft_above',
       'sqft_basement', 'sqft_garage', 'sqft_patio', 'yr_built',
       'yr_renovated', 'address', 'lat', 'long'],
      dtype='object')

In [5]:
hd.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront        object
greenbelt         object
nuisance          object
view              object
condition         object
grade             object
heat_source       object
sewer_system      object
sqft_above         int64
sqft_basement      int64
sqft_garage        int64
sqft_patio         int64
yr_built           int64
yr_renovated       int64
address           object
lat              float64
long             float64
dtype: object

Some of these columns, such as view, condition, and grade, seem to be ordinal numerical columns, but are not currently stored as the type of variable we will ultimately want them to be to include them in our analysis. Others, such as waterfront, greenbelt, and nuisance, may be good candidates for one-hot encoding. We can dig into these further when we begin processing our data.

In [6]:
# describe the characteristics of the continuous variables in the dataset
hd.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,sqft_basement,sqft_garage,sqft_patio,yr_built,yr_renovated,lat,long
count,3.015500e+04,3.015500e+04,30155.000000,30155.000000,30155.000000,3.015500e+04,30155.000000,30155.000000,30155.000000,30155.000000,30155.000000,30155.000000,30155.000000,30155.000000,30155.000000
mean,4.538104e+09,1.108536e+06,3.413530,2.334737,2112.424739,1.672360e+04,1.543492,1809.826098,476.039396,330.211142,217.412038,1975.163953,90.922301,47.328076,-121.317397
std,2.882587e+09,8.963857e+05,0.981612,0.889556,974.044318,6.038260e+04,0.567717,878.306131,579.631302,285.770536,245.302792,32.067362,416.473038,1.434005,5.725475
min,1.000055e+06,2.736000e+04,0.000000,0.000000,3.000000,4.020000e+02,1.000000,2.000000,0.000000,0.000000,0.000000,1900.000000,0.000000,21.274240,-157.791480
25%,2.064175e+09,6.480000e+05,3.000000,2.000000,1420.000000,4.850000e+03,1.000000,1180.000000,0.000000,0.000000,40.000000,1953.000000,0.000000,47.405320,-122.326045
50%,3.874011e+09,8.600000e+05,3.000000,2.500000,1920.000000,7.480000e+03,1.500000,1560.000000,0.000000,400.000000,150.000000,1977.000000,0.000000,47.551380,-122.225585
75%,7.287100e+09,1.300000e+06,4.000000,3.000000,2619.500000,1.057900e+04,2.000000,2270.000000,940.000000,510.000000,320.000000,2003.000000,0.000000,47.669913,-122.116205
max,9.904000e+09,3.075000e+07,13.000000,10.500000,15360.000000,3.253932e+06,4.000000,12660.000000,8020.000000,3580.000000,4370.000000,2022.000000,2022.000000,64.824070,-70.074340


From this initial examination it seems like there may be at least a few data imputation errors - it's unlikely that there is a home with only 3 square feet of living space, for example. We'll need to make sure to deal with those issues before beginning to build our model. We should also check for null values.

In [7]:
hd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30155 entries, 0 to 30154
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             30155 non-null  int64  
 1   date           30155 non-null  object 
 2   price          30155 non-null  float64
 3   bedrooms       30155 non-null  int64  
 4   bathrooms      30155 non-null  float64
 5   sqft_living    30155 non-null  int64  
 6   sqft_lot       30155 non-null  int64  
 7   floors         30155 non-null  float64
 8   waterfront     30155 non-null  object 
 9   greenbelt      30155 non-null  object 
 10  nuisance       30155 non-null  object 
 11  view           30155 non-null  object 
 12  condition      30155 non-null  object 
 13  grade          30155 non-null  object 
 14  heat_source    30123 non-null  object 
 15  sewer_system   30141 non-null  object 
 16  sqft_above     30155 non-null  int64  
 17  sqft_basement  30155 non-null  int64  
 18  sqft_g

Fortunately, there are very few null values in our dataset - only 32 in heat_source, and 14 in sewer_system. We can wait until we begin to process our data to deal with those.

## Data Preparation

Describe and justify the process for preparing the data for analysis.

Questions to consider:

- Were there variables you dropped or created?
- How did you address missing values or outliers?
- Why are these choices appropriate given the data and the business problem?

Let's start by dealing with the row for the home that is only 3 square feet.

In [8]:
hd[hd['sqft_living'] <= 100]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,greenbelt,...,sewer_system,sqft_above,sqft_basement,sqft_garage,sqft_patio,yr_built,yr_renovated,address,lat,long
14977,1549500215,12/17/2021,1803000.0,4,4.0,3,326701,2.0,NO,NO,...,PRIVATE,2,1,1,0,2021,0,"31072 Northeast 165th Street, Duvall, Washingt...",47.745290,-121.92362
28816,2923099027,12/13/2021,205000.0,1,0.0,100,25400,1.0,YES,NO,...,NaN,100,0,0,140,1963,0,"49622 172nd Street Southeast, North Bend, Wash...",47.444745,-121.67879


There are two listings for homes in the dataset that are 100 square feet or smaller. The second of these is an oddity but there aren't enough issues to suggest it is included incorrectly in the dataset, whereas the first one is almost certainly an incorrect listing. Let's remove it.

In [9]:
hd.drop(index = 14977, axis = 0, inplace = True)

In building our model, we'll also want to exclude sale listings for homes with no bedrooms or no bathrooms. Our stakeholder (families moving to the area) are not interested in building a home on an empty lot, or making major renovations to prepare it for them to live in - they need a house that's ready for them (even if it's in need of some other work).

In [10]:
# find and drop sales records with no bedrooms or no bathrooms
nobed_nobath = hd.loc[(hd['bedrooms'] == 0) | (hd['bathrooms'] == 0)]
hd.drop(index = nobed_nobath.index, inplace = True, axis = 0)

We'll also want to add a few columns with basic information that isn't already included in the dataset, such as zip code and age of the house.

In [11]:
# add zip column for zipcodes
hd["zip"] = hd['address'].str[-20:-15].astype(int)

# add column for years old and sale year
hd["years_old"] = 2023 - hd["yr_built"]
hd["sale_year"] = hd['date'].str[-4:].astype(int)

Now that we have zip codes for each listing, we'll need to confirm that the dataset includes only homes in King County. To do so, we can create a list of zip codes using the [King County GIS Open Data portal](https://gis-kingcounty.opendata.arcgis.com/datasets/kingcounty::zipcodes-for-king-county-and-surrounding-area-zipcode-area/explore?filters=eyJDT1VOVFlfTkFNRSI6WyJLaW5nIENvdW50eSJdfQ%3D%3D&location=47.503077%2C-121.485100%2C8.71&showTable=true) and compare the zip codes for our records to that list to remove any that aren't in the county.

In [12]:
# create a list of King County zip codes
kingcounty_zips=[98001,98002,98003,98004,98005,98006,98007,98008,
                 98010,98011,98014,98019,98021,98022,98023,98024,
                 98027,98028,98029,98030,98031,98032,98033,98034,
                 98038,98039,98040,98042,98045,98047,98050,98051,
                 98052,98053,98055,98056,98057,98058,98059,98065,
                 98068,98070,98072,98074,98075,98077,98092,98101,
                 98102,98103,98104,98105,98106,98107,98108,98109,
                 98112,98115,98116,98117,98118,98119,98121,98122,
                 98125,98126,98133,98134,98136,98144,98146,98148,
                 98154,98155,98158,98164,98166,98168,98177,98178,
                 98188,98195,98198,98199,98224,98354,98422]

# find rows that have zip codes outside of King County
non_kc_zips = hd.loc[~hd["zip"].isin(kingcounty_zips)]
non_kc_zips.shape

(959, 28)

It looks like 959 of our records are from places other than King County. Let's remove them from our dataset.

In [13]:
# drop rows with zips that are not in King County
hd.drop(index = non_kc_zips.index, inplace = True, axis = 0)
hd

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,greenbelt,...,sqft_garage,sqft_patio,yr_built,yr_renovated,address,lat,long,zip,years_old,sale_year
0,7399300360,5/24/2022,675000.0,4,1.0,1180,7140,1.0,NO,NO,...,0,40,1969,0,"2102 Southeast 21st Court, Renton, Washington ...",47.461975,-122.19052,98055,54,2022
1,8910500230,12/13/2021,920000.0,5,2.5,2770,6703,1.0,NO,NO,...,0,240,1950,0,"11231 Greenwood Avenue North, Seattle, Washing...",47.711525,-122.35591,98133,73,2021
2,1180000275,9/29/2021,311000.0,6,2.0,2880,6156,1.0,NO,NO,...,0,0,1956,0,"8504 South 113th Street, Seattle, Washington 9...",47.502045,-122.22520,98178,67,2021
3,1604601802,12/14/2021,775000.0,3,3.0,2160,1400,2.0,NO,NO,...,200,270,2010,0,"4079 Letitia Avenue South, Seattle, Washington...",47.566110,-122.29020,98118,13,2021
4,8562780790,8/24/2021,592500.0,2,2.0,1120,758,2.0,NO,NO,...,550,30,2012,0,"2193 Northwest Talus Drive, Issaquah, Washingt...",47.532470,-122.07188,98027,11,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30150,7834800180,11/30/2021,1555000.0,5,2.0,1910,4000,1.5,NO,NO,...,0,210,1921,0,"4673 Eastern Avenue North, Seattle, Washington...",47.664740,-122.32940,98103,102,2021
30151,194000695,6/16/2021,1313000.0,3,2.0,2020,5800,2.0,NO,NO,...,0,520,2011,0,"4131 44th Avenue Southwest, Seattle, Washingto...",47.565610,-122.38851,98116,12,2021
30152,7960100080,5/27/2022,800000.0,3,2.0,1620,3600,1.0,NO,NO,...,240,110,1995,0,"910 Martin Luther King Jr Way, Seattle, Washin...",47.610395,-122.29585,98122,28,2022
30153,2781280080,2/24/2022,775000.0,3,2.5,2570,2889,2.0,NO,NO,...,480,100,2006,0,"17127 114th Avenue Southeast, Renton, Washingt...",47.449490,-122.18908,98055,17,2022


Let's do a quick check for duplicate values in the 'id' column, since that should be a unique identifier for each listing.

In [14]:
# check to see if there's any values greater than 1
hd['id'].value_counts()

1233100736    2
6632900607    1
3678900411    1
5437810310    1
7852180850    1
             ..
2251500240    1
5253300275    1
5149300370    1
1972202200    1
9430110210    1
Name: id, Length: 29144, dtype: int64

Looks like there's one duplicate listing - let's double check, and if it is a duplicate let's remove it.

In [15]:
# check information for rows with duplicate id values
hd[hd['id'] == 1233100736]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,greenbelt,...,sqft_garage,sqft_patio,yr_built,yr_renovated,address,lat,long,zip,years_old,sale_year
4845,1233100736,9/28/2021,2600000.0,3,4.0,3500,8455,2.0,NO,NO,...,500,290,2019,0,"12611 Northeast 87th Place, Kirkland, Washingt...",47.68069,-122.172135,98033,4,2021
4846,1233100736,9/28/2021,2600000.0,3,4.0,3500,8455,2.0,NO,NO,...,500,290,2019,0,"12611 Northeast 87th Place, Kirkland, Washingt...",47.68069,-122.172135,98033,4,2021


Those listings are identical, so this is almost certainly an accidental duplicate. Let's remove it.

In [16]:
# remove the duplicate based on id
hd = hd.drop_duplicates(subset=["id"])

# check that it's gone 
hd[hd['id'] == 1233100736]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,greenbelt,...,sqft_garage,sqft_patio,yr_built,yr_renovated,address,lat,long,zip,years_old,sale_year
4845,1233100736,9/28/2021,2600000.0,3,4.0,3500,8455,2.0,NO,NO,...,500,290,2019,0,"12611 Northeast 87th Place, Kirkland, Washingt...",47.68069,-122.172135,98033,4,2021


Now that we've done some basic cleaning and feature additions, let's take another look at the description for our data.

In [24]:
hd.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,sqft_basement,sqft_garage,sqft_patio,yr_built,yr_renovated,lat,long,zip,years_old,sale_year
count,2.914400e+04,2.914400e+04,29144.000000,29144.000000,29144.000000,2.914400e+04,29144.000000,29144.000000,29144.000000,29144.000000,29144.000000,29144.000000,29144.000000,29144.000000,29144.000000,29144.000000,29144.000000,29144.000000
mean,4.535657e+09,1.112903e+06,3.441017,2.335506,2132.796974,1.711110e+04,1.516110,1827.234559,481.547488,336.970972,216.548003,1973.842232,93.527724,47.542959,-122.214876,98077.313684,49.157768,2021.361344
std,2.882395e+09,8.928455e+05,0.970070,0.892642,975.878972,6.059726e+04,0.547403,880.042425,586.098105,286.118703,246.597334,31.657073,422.109515,0.146098,0.141526,54.482404,31.657073,0.480398
min,1.000055e+06,2.736000e+04,1.000000,0.500000,290.000000,4.020000e+02,1.000000,290.000000,0.000000,0.000000,0.000000,1900.000000,0.000000,47.159700,-122.526650,98001.000000,1.000000,2021.000000
25%,2.085201e+09,6.450000e+05,3.000000,2.000000,1440.000000,5.000000e+03,1.000000,1190.000000,0.000000,0.000000,40.000000,1952.000000,0.000000,47.429599,-122.328283,98032.000000,22.000000,2021.000000
50%,3.869900e+09,8.686500e+05,3.000000,2.500000,1950.000000,7.560000e+03,1.500000,1580.000000,0.000000,400.000000,142.000000,1976.000000,0.000000,47.557428,-122.234287,98059.000000,47.000000,2021.000000
75%,7.286500e+09,1.310000e+06,4.000000,3.000000,2640.000000,1.077300e+04,2.000000,2290.000000,960.000000,520.000000,310.000000,2001.000000,0.000000,47.671695,-122.128011,98117.000000,71.000000,2022.000000
max,9.904000e+09,3.075000e+07,13.000000,10.500000,15360.000000,3.253932e+06,4.000000,12660.000000,8020.000000,3580.000000,4370.000000,2022.000000,2022.000000,47.789830,-121.417590,98422.000000,123.000000,2022.000000


There are still a wide range of housing types in our dataset, from homes with one bedroom that are only a few hundred feet, to mega-mansions with 13 bedrooms and over 15,000 square feet of space. While we will want to eventually hone in on the target market for our clients, it will be useful to build our model on the largest dataset we can to ensure our price predictions are as accurate as possible.

## Modeling

Describe and justify the process for analyzing or modeling the data.

Questions to consider:

- How did you analyze the data to arrive at an initial approach?
- How did you iterate on your initial approach to make it better?
- Why are these choices appropriate given the data and the business problem?

## Evaluation

The evaluation of each model should accompany the creation of each model, and you should be sure to evaluate your models consistently.

Evaluate how well your work solves the stated business problem. 

Questions to consider:

- How do you interpret the results?
- How well does your model fit your data? How much better is this than your baseline model? Is it over or under fit?
- How well does your model/data fit any modeling assumptions?

For the final model, you might also consider:

- How confident are you that your results would generalize beyond the data you have?
- How confident are you that this model would benefit the business if put into use?

### Baseline Understanding

- What does a baseline, model-less prediction look like?

In [17]:
# code here to arrive at a baseline prediction

### First  Model

Before going too far down the data preparation rabbit hole, be sure to check your work against a first 'substandard' model! What is the easiest way for you to find out how hard your problem is?

In [18]:
# code here for your first 'substandard' model

In [19]:
# code here to evaluate your first 'substandard' model

### Modeling Iterations

Now you can start to use the results of your first model to iterate - there are many options!

In [20]:
# code here to iteratively improve your models

In [21]:
# code here to evaluate your iterations

### 'Final' Model

In the end, you'll arrive at a 'final' model - aka the one you'll use to make your recommendations/conclusions. This likely blends any group work. It might not be the one with the highest scores, but instead might be considered 'final' or 'best' for other reasons.

In [22]:
# code here to show your final model

In [23]:
# code here to evaluate your final model

## Conclusions

Provide your conclusions about the work you've done, including any limitations or next steps.

Questions to consider:

- What would you recommend the business do as a result of this work?
- What are some reasons why your analysis might not fully solve the business problem?
- What else could you do in the future to improve this project (future work)?
